# Poles as Modes, Zeros as Cancellations — and When FIR Zeros Lie on the Unit Circle

This notebook adds the **physical and mathematical interpretation** of poles and zeros:

- Continuous-time modes: \(e^{st}\)  
- Discrete-time modes: \(p^n\)  
- How poles encode decay, growth, and oscillation  
- How zeros encode cancellation and spectral nulls  
- Whether FIR filters *require* zeros on the unit circle (short answer: **no**)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_zplane(poles=None, zeros=None, title="z-plane"):
    theta = np.linspace(0, 2*np.pi, 400)
    unit_x = np.cos(theta)
    unit_y = np.sin(theta)

    fig, ax = plt.subplots(figsize=(5,5))
    ax.axhline(0, linewidth=1)
    ax.axvline(0, linewidth=1)
    ax.plot(unit_x, unit_y)
    
    if zeros is not None and len(zeros) > 0:
        ax.plot(np.real(zeros), np.imag(zeros), 'o', label="zeros")
    if poles is not None and len(poles) > 0:
        ax.plot(np.real(poles), np.imag(poles), 'x', markersize=10, label="poles")

    ax.set_aspect('equal', 'box')
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_title(title)
    ax.set_xlabel("Re{z}")
    ax.set_ylabel("Im{z}")
    ax.legend()
    plt.show()


## 1) Poles as natural modes of the system

Continuous-time pole at \(s=s_0\) → mode \(e^{s_0 t}\)  
Discrete-time pole at \(z=p\) → mode \(p^n\)

Write \(p = r e^{j\omega_0}\):

\[
p^n = r^n e^{j\omega_0 n}
\]

- \(r<1\): decay  
- \(r=1\): sustained oscillation  
- \(r>1\): growth / instability  


In [ ]:
# Example poles and modes
poles = np.array([0.8, np.exp(1j*0.5), 1.1])

plot_zplane(poles=poles, zeros=[], title="Three poles: decaying, oscillatory, unstable")

N = 50
n = np.arange(N)

fig, ax = plt.subplots(figsize=(8,4))
for p in poles:
    h = (p**n).real
    ax.plot(n, h, label=f"p = {p:.2f}")
ax.set_title("Modes p^n for different pole radii")
ax.set_xlabel("n")
ax.set_ylabel("p^n (real part)")
ax.legend()
plt.show()


## 2) Zeros and spectral nulls

Zero at $$(z=z_0) ⇒ (H(z_0)=0).$$  
If $$z_0=e^{j\omega_0}$$ lies on the unit circle ⇒ complete notch at $$\omega_0$$.


In [ ]:
# Notch example
w0 = 0.6*np.pi
zeros = np.array([np.exp(1j*w0)])
plot_zplane(poles=[], zeros=zeros, title="Zero on unit circle (notch)")

w = np.linspace(0, np.pi*2, 512)
z = np.exp(1j*w)
H = 1 - np.exp(-1j*w0)*z**-1

fig, ax = plt.subplots(figsize=(7,4))
ax.plot(w, np.abs(H))
ax.set_title("Notch filter magnitude response")
ax.set_xlabel("ω")
ax.set_ylabel("|H|")
plt.show()


## 3) Do FIR filters require zeros on the unit circle?

**No.**

An FIR filter is any polynomial in \(z^{-1}\):

\[
H(z) = b_0 + b_1 z^{-1} + \cdots + b_M z^{-M}
\]

Zeros may lie anywhere in the z-plane.

Zeros on the unit circle occur when you *want* exact spectral nulls or linear-phase structure.


### Example: FIR with zeros not on the unit circle

In [ ]:
b = np.array([1.0, -0.4, 0.2])
zeros = np.roots(b[::-1])

plot_zplane(poles=[], zeros=zeros, title="FIR zeros not on unit circle")

w = np.linspace(0, np.pi, 512)
z = np.exp(1j*w)
H = b[0] + b[1]*z**-1 + b[2]*z**-2

fig, ax = plt.subplots(figsize=(7,4))
ax.plot(w, np.abs(H))
ax.set_title("FIR magnitude response (no exact nulls)")
ax.set_xlabel("ω")
ax.set_ylabel("|H|")
plt.show()


### Example: linear-phase FIR with unit-circle zeros

In [ ]:
b = np.array([1, 0, -2, 0, 1])  # symmetric FIR
zeros = np.roots(b[::-1])

plot_zplane(poles=[], zeros=zeros, title="Linear-phase FIR zeros")

w = np.linspace(0, np.pi, 512)
z = np.exp(1j*w)
H = sum(b[k]*z**(-k) for k in range(len(b)))

fig, ax = plt.subplots(figsize=(7,4))
ax.plot(w, np.abs(H))
ax.set_title("Linear-phase FIR magnitude response")
ax.set_xlabel("ω")
ax.set_ylabel("|H|")
plt.show()


## 4) Final takeaways

- Poles correspond to **natural modes** \(p^n\)  
- Radius = decay/growth, angle = frequency  
- Zeros correspond to **cancelled modes / spectral nulls**  
- FIR filters do **not** require unit-circle zeros  
- Unit-circle zeros are a **design choice**, not a structural necessity  
- IIR poles create infinite impulse responses via feedback  
